<a href="https://colab.research.google.com/github/ShuYuHuang/nlp4aml/blob/master/prep_5kdata_import.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('~/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /root/drive


In [2]:
# coding=utf-8
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json

In [3]:
file_loc='/root/drive/My Drive/ColabData/'
data_pd=pd.read_csv(file_loc+'tbrain_train_final_0610.csv')
def get_homepage(site):
  cnt=0
  begin=0
  for ee, ww in enumerate(site):
      if ww=='/':
        cnt+=1
        if cnt==2:
          begin=ee+1
        elif cnt==3:
          return site[begin:ee]
  return 0
data_pd['homepage']=data_pd['hyperlink'].apply(lambda w: get_homepage(w))

In [4]:
data_pd.sample(3)

,news_ID,hyperlink,content,name,homepage
3494,3495,https://udn.com/news/story/6811/4055559,FedEx調降獲利展望 ### 省略內文 ### 已成為貨運競爭對手。,[],udn.com
1946,1947,https://www.chinatimes.com/realtimenews/201910...,台南市南化烏山風景區著名景點「龍虎寺」 ### 省略內文 ### 請與玉井偵查隊06-574...,[],www.chinatimes.com
3923,3924,https://news.mingpao.com/ins/%e6%b8%af%e8%81%9...,(12:55) prevnext 34歲男護士今年5月涉於荃灣港安醫院非禮一名女子 ### ...,[],news.mingpao.com


In [7]:
data_pd['homepage'].value_counts()

news.cnyes.com              709
www.mirrormedia.mg          626
www.chinatimes.com          621
m.ctee.com.tw               530
udn.com                     448
news.mingpao.com            379
news.ltn.com.tw             332
mops.twse.com.tw            273
technews.tw                 128
www.businesstoday.com.tw    120
www.ettoday.net             118
www.hk01.com                117
news.tvbs.com.tw            112
sina.com.hk                  72
www.bnext.com.tw             59
hk.on.cc                     57
finance.technews.tw          56
www.cw.com.tw                45
domestic.judicial.gov.tw     30
www.wealth.com.tw            29
tw.news.yahoo.com            27
www.coolloud.org.tw          25
www.setn.com                 16
www.managertoday.com.tw      14
www.hbrtaiwan.com            14
m.ltn.com.tw                 12
ccc.technews.tw              10
www.fsc.gov.tw                9
money.udn.com                 8
ec.ltn.com.tw                 6
www.cna.com.tw                6
ol.mingp

In [224]:
display(data_pd[data_pd['homepage']=='estate.ltn.com.tw'][:])
display(data_pd['hyperlink'][502])

,news_ID,hyperlink,content,name,homepage
502,503,http://estate.ltn.com.tw/article/6814,位於桃園市桃園區春日路的建案「銘曜高尚」近日驚傳建商落跑 ### 省略內文 ### 才能確保...,"['邱俊銘', '邱水成', '邱秀芬']",estate.ltn.com.tw


'http://estate.ltn.com.tw/article/6814'

In [ ]:
def extract_content(ii):  
  global data_pd
  r = requests.get(data_pd['hyperlink'][ii])
  # Create soup from content of request
  c = r.content
  soup = BeautifulSoup(c)
  warns=False
  try:
    txt="<h1>"+soup.title.text+"</h1>"
    
  #####################Stage 2 ###########################################
    elif data_pd['homepage'][ii]=='www.bnext.com.tw'':
      for aa in soup.find("article",attrs={"class":"main_content"}).findAll('p'):
        txt+=aa.text
      txt+="<time>"+soup.find("span",attrs={"class":"item"}).text+"</time>" 

    else:
      warns=True

    txt+="<aml>"+data_pd['name'][ii]+"</aml>"
    if warns==False:
      file1 = open(f"{file_loc}5k_news_data/{data_pd['news_ID'][ii]}.txt","w")#write mode 
      file1.write(txt) 
      file1.close()
  except:
    file1 = open(f"{file_loc}5k_news_data/{data_pd['news_ID'][ii]}.txt","w")#write mode 
    file1.write("404-error") 
    file1.close()

In [226]:

def extract_content(ii):
  global data_pd
  r = requests.get(data_pd['hyperlink'][ii])
  # Create soup from content of request
  c = r.content
  soup = BeautifulSoup(c)
  warns=False
  homepage=data_pd['homepage'][ii]
  try:
    txt="<h1>"+soup.title.text+"</h1>"
    #####################Dat 1 progress ###########################################
    if data_pd['homepage'][ii]=='news.cnyes.com':
      txt+=soup.find('section').text
      txt+="<time>"+soup.find('time').text+"</time>"
  
    elif data_pd['homepage'][ii]=='www.mirrormedia.mg':
      for aa in soup.find_all('p'):
        txt+=aa.text
      txt+="<time>"+soup.find('div', attrs={"class":"date"}).text+"</time>"

    elif data_pd['homepage'][ii]=="www.chinatimes.com":
      txt+=soup.find('div', attrs={"class":"article-body"}).text
      txt+="<time>"+soup.find('time')['datetime']+"</time>"

    elif data_pd['homepage'][ii]=="m.ctee.com.tw":
      txt+=soup.find('div', attrs={"class":"entry-content"}).text
      txt+="<time>"+soup.find('div',attrs={"class":"post-meta-date"}).text+"</time>"

    elif data_pd['homepage'][ii]=="udn.com":
      txt+=soup.find('section',attrs={'class':'article-content__editor '}).text
      txt+="<time>"+soup.find('time').text+"</time>"

    elif data_pd['homepage'][ii]=="news.mingpao.com":
      txt+=soup.find('article').text
      txt+="<time>"+soup.find('div',attrs={'itemprop':'datePublished'}).text+"</time>"

    elif data_pd['homepage'][ii]=="news.ltn.com.tw":
      for aa in soup.find('div',attrs={'class':'text boxTitle boxText'}).find_all('p'):
        if not aa.has_attr('class'):
          txt+=aa.text
      txt+="<time>"+soup.find('span',attrs={'class':'time'}).text+"</time>"

    elif data_pd['homepage'][ii]=="mops.twse.com.tw":
      tmp=soup.find_all('td', attrs={"style":"text-align:left !important;"})
      txt="<h1>"+tmp[6].text+"</h1>"+tmp[9].text
      txt+="<time>"+tmp[1].text+"</time>"

    elif  data_pd['homepage'][ii]=="technews.tw":
      for aa in soup.find('div',class_="indent").find_all('p')[:-1]:
        txt+=aa.text
      txt+="<time>"+soup.find_all('span', attrs={"body"})[1].text+"</time>"

    elif  data_pd['homepage'][ii]=="www.businesstoday.com.tw":
      txt+=soup.find('div', attrs={"itemprop":"articleBody"}).text  
      txt+="<time>"+soup.find("meta",attrs={"property":"article:published_time"})['content']+"</time>"

    elif  data_pd['homepage'][ii]=="www.ettoday.net":
      for aa in soup.find("div",attrs={"class":"content-container"}).find_all("p")[3:]:
        if aa.text!='請繼續往下閱讀...':
          txt+=aa.text
      txt+="<time>"+soup.find("time",attrs={"class":"header-time"})['datetime']+"</time>" 

    elif data_pd['homepage'][ii]=='www.hk01.com':
      for aa in soup.findAll("p")[2:]:
        txt+=aa.text
      txt+="<time>"+soup.time.text+"</time>" 

    elif data_pd['homepage'][ii]=='news.tvbs.com.tw':
      for aa in AA.findAll('br')[::2]:
        txt+=aa.previous_element
      txt+="<time>"+soup.find("meta",attrs={"name":"pubdate"})["content"]+"</time>" 

    elif data_pd['homepage'][ii]=='sina.com.hk':
      for aa in soup.find("div",attrs={"class":"news-body"}).findAll("p")[:-5]:
        txt+=aa.text
      txt+="<time>"+soup.find("div",attrs={"class":"news-datetime"}).text+"</time>" 
    #####################Day 2 progress###########################################
    if homepage=='www.bnext.com.tw':
      for aa in soup.find("article",attrs={"class":"main_content"}).findAll('p'):
        txt+=aa.text
      txt+="<time>"+soup.find("span",attrs={"class":"item"}).text+"</time>" 

    elif homepage=='hk.on.cc':
      for aa in soup.findAll("div",attrs={"class":"paragraph"}):
        txt+=aa.text
      txt+="<time>"+soup.find("span",attrs={"class":"datetime"}).text+"</time>" 
      
    elif homepage=='finance.technews.tw':
      for aa in soup.find("div",attrs={"class":"indent"}).findAll("p"):
        txt+=aa.text
      for aa in soup.find_all("span",attrs={"class":"body"}):
        if aa.findPrevious("span",class_="head").text=="發布日期":
          txt+="<time>"+aa.text+"</time>" 

    elif homepage=='www.cw.com.tw':
      txt="<h1>"+soup.h1.text+"</h1>"
      for aa in soup.findAll('p')[3:-11]:
        txt+=aa.text
      txt+="<time>"+soup.time.text+"</time>" 

    elif homepage=='www.wealth.com.tw':
      txt+=soup.find("div",attrs={"itemprop":"articleBody"}).text
      txt+="<time>"+soup.find("meta",attrs={"property":"article:published_time"})['content']+"</time>"

    elif homepage=='tw.news.yahoo.com':
      for aa in soup.find('div',attrs={"class":"caas-body"}).findAll('p')[:-6]:
        txt+=aa.text
      txt+="<time>"+soup.time.text+"</time>" 

    elif homepage=='www.coolloud.org.tw':
      txt+=soup.find('div',attrs={"class":"field field-name-body field-type-text-with-summary field-label-hidden"}).text
      txt+="<time>"+soup.find("span",attrs={"class":"date-display-single"})+"</time>" 
    
    elif homepage=='www.setn.com':
      txt+=soup.article.text
      txt+="<time>"+soup.find("time",class_="page-date").text+"</time>" 

    elif homepage=='www.managertoday.com.tw':
      for aa in soup.article.findAll("p"):
        txt+=aa.text
      txt+="<time>"+soup.article.find("time").text+"</time>" 

    elif homepage=='www.hbrtaiwan.com':
      for aa in soup.find("div",attrs={"class":"article article-first-row premium"}).findAll("p"):
        txt+=aa.text
      matches= re.compile(r"datePublished\":\ \".*\"").finditer(soup.prettify())
      for mm in matches:
        txt+="<time>"+mm.group(0)[17:-1]+"</time>"
        break

    elif homepage=='m.ltn.com.tw':
      for aa in soup.find("div",attrs={"data-desc":"內文"}).findAll("p"):
        if not aa.has_attr('class'):
          txt+=aa.text
      matches= re.compile(r"datePublished\":\ \".*\"").finditer(soup.prettify())
      for mm in matches:
        txt+="<time>"+mm.group(0)[17:-1]+"</time>"
        break

    elif homepage=='ccc.technews.tw':
      for aa in soup.find("div",attrs={"class":"indent"}).findAll("p"):
        if not aa.has_attr('class'):
          txt+=aa.text
      for aa in soup.find_all("span",attrs={"class":"body"}):
        if aa.findPrevious("span",class_="head").text=="發布日期":
          txt+="<time>"+aa.text+"</time>"
          break
        
    elif homepage=='www.fsc.gov.tw':
      for aa in soup.find("div",attrs={"class":"indent"}).findAll("p"):
        if not aa.has_attr('class'):
          txt+=aa.text
      txt="</time>"+soup.find("div",class_="contentdate").txt+"</time>"

    elif homepage=='money.udn.com':
      for aa in soup.find("div",attrs={"id":"article_body"}).findAll("p"):
          txt+=aa.text
      matches= re.compile(r"datePublished\":\ \".*\"").finditer(soup.prettify())
      for mm in matches:
        txt+="<time>"+mm.group(0)[17:-1]+"</time>"
        break

    elif homepage=='ec.ltn.com.tw':
      for aa in soup.find("div",attrs={"data-desc":"內文"}).findAll("p"):
        if not aa.has_attr('class'):
          txt+=aa.text
      matches= re.compile(r"datePublished\":\ \".*\"").finditer(soup.prettify())
      for mm in matches:
        txt+="<time>"+mm.group(0)[17:-1]+"</time>"
        break
    
    elif homepage=='www.cna.com.tw':
      for aa in soup.findAll("p"):
        txt+=aa.text
      txt+="<time>"+soup.find("div",class_="updatetime").span.text+"</time>"
    
    elif homepage=='ol.mingpao.com':
      for aa in soup.findAll("p"):
        txt+=aa.text
      txt+="<time>"+soup.h5.text[5:]+"</time>"

    elif homepage=='www.nextmag.com.tw':
      for aa in soup.find("div",class_="article-content").findAll("p")[1:]:
        txt+=aa.text
      matches= re.compile(r"datePublished\":\ \".*\"").finditer(soup.prettify())
      for mm in matches:
        txt+="<time>"+mm.group(0)[17:-1]+"</time>"
        break

    elif homepage=='ent.ltn.com.tw':
      for aa in soup.findAll("p"):
        if not aa.has_attr('class'):
          txt+=aa.text
      matches= re.compile(r"datePublished\":\ \".*\"").finditer(soup.prettify())
      for mm in matches:
        txt+="<time>"+mm.group(0)[17:-1]+"</time>"
        break
    
    elif homepage=='news.ebc.net.tw':
      for aa in soup.find("div",attrs={"class":"raw-style"}).findAll("p"):
        if not aa.has_attr('class'):
          txt+=aa.text
      matches= re.compile(r"datePublished\":\ \".*\"").finditer(soup.prettify())
      for mm in matches:
        txt+="<time>"+mm.group(0)[17:-1]+"</time>"
        break
    
    elif homepage=='www.nownews.com':
      txt+=soup.find("div",attrs={"itemprop":"articleBody"}).text
      matches= re.compile(r"datePublished\":\ \".*\"").finditer(soup.prettify())
      for mm in matches:
        txt+="<time>"+mm.group(0)[17:-1]+"</time>"
        break
    
    elif homepage=='www.storm.mg':
      txt+=soup.find("div",attrs={"itemprop":"articleBody"}).text
      txt+=soup.find("meta",attrs={"name":"publishedTime"})["content"]
    
        
    elif homepage=='house.ettoday.net':
      txt+=soup.find("div",attrs={"itemprop":"articleBody"}).text
      txt+=soup.find("meta",attrs={"name":"pubdate"})["content"]

    elif homepage=='estate.ltn.com.tw':
      txt+=soup.find("div",attrs={"itemprop":"articleBody"}).text
      txt+=soup.find("meta",attrs={"name":"pubdate"})["content"]

    else:
      warns=True

    txt+="<aml>"+data_pd['name'][ii]+"</aml>"
    if warns==False:
      file1 = open(f"{file_loc}5k_news_data/{data_pd['news_ID'][ii]}.txt","w")#write mode 
      file1.write(txt) 
      file1.close()
  except:
    file1 = open(f"{file_loc}5k_news_data/{data_pd['news_ID'][ii]}.txt","w")#write mode 
    file1.write("404-error") 
    file1.close()

In [231]:
import time
for ii in data_pd.index[24:]:
  if ii>0:
    if data_pd['homepage'][ii]==data_pd['homepage'][ii-1]:
      time.sleep(1)
  extract_content(ii)